### Chuẩn bị dữ liệu

In [60]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
df = pd.read_csv('./data/train.csv', sep='|')
df_test = pd.read_csv('./data/test.csv', sep='|')

new_df = df.dropna()
new_df_test = df_test.dropna()

new_df.insert(1, 'totalItemsScanned', new_df['scannedLineItemsPerSecond'] * new_df['totalScanTimeInSeconds'])
new_df_test.insert(1, 'totalItemsScanned', new_df_test['scannedLineItemsPerSecond'] * new_df_test['totalScanTimeInSeconds'])

y_train = new_df.pop('fraud')

X_train = new_df.drop(columns=[
    'scannedLineItemsPerSecond', 'lineItemVoidsPerPosition', 'valuePerSecond',
    'quantityModifications', 'grandTotal'
])
X_test = new_df_test.drop(columns=[
    'scannedLineItemsPerSecond', 'lineItemVoidsPerPosition', 'valuePerSecond',
    'quantityModifications', 'grandTotal'
])

print('Labels counts in X_train',len(X_train))
print('Total rows in test set',len(X_test))

Labels counts in X_train 1879
Total rows in test set 498121


**Chia tập dữ liệu**

In [61]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.7, random_state=1)

**Thực hiện gọi áp dụng mô hình chưa sử dụng siêu tham số**

In [62]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression


# Dự đoán kết quả từng mô hình và lưu vào DataFrame
# Decision Tree
dt_model = DecisionTreeClassifier(max_depth=3, criterion='gini')
dt_model.fit(X_train, y_train)
dt_preds_train = dt_model.predict(X_train)
dt_preds_val = dt_model.predict(X_val)

# K-Nearest Neighbors
knn_model = KNeighborsClassifier()
knn_model.fit(X_train, y_train)
knn_preds_train = knn_model.predict(X_train)
knn_preds_val = knn_model.predict(X_val)

# Logistic Regression
lr_model = LogisticRegression(max_iter=100000 )  # Tăng số vòng lặp tối đa
lr_model.fit(X_train, y_train)
lr_preds_train = lr_model.predict(X_train)
lr_preds_val = lr_model.predict(X_val)

### **Đánh giá hiệu suất**

In [63]:
import sklearn.metrics as metrics

dt_train_cm = metrics.confusion_matrix(y_train, dt_preds_train)
dt_train_accuracy = metrics.accuracy_score(y_train,dt_preds_train)
print('The train confusion matrix for Decision Tree model is:', '\n', dt_train_cm)

dt_val_cm = metrics.confusion_matrix(y_val,dt_preds_val)
dt_val_accuracy = metrics.accuracy_score(y_val,dt_preds_val)
print('The validation confusion matrix for Decision Tree model is:', '\n', dt_val_cm)

print('Train accuracy: ',dt_train_accuracy,' | Validation accuracy: ',dt_val_accuracy)

The train confusion matrix for Decision Tree model is: 
 [[521   4]
 [  5  33]]
The validation confusion matrix for Decision Tree model is: 
 [[1239   11]
 [  21   45]]
Train accuracy:  0.9840142095914742  | Validation accuracy:  0.9756838905775076


In [64]:
knn_train_cm = metrics.confusion_matrix(y_train, knn_preds_train)
knn_train_accuracy = metrics.accuracy_score(y_train,knn_preds_train)
print('The train confusion matrix for K-NN model is:', '\n', knn_train_cm)

knn_val_cm = metrics.confusion_matrix(y_val,knn_preds_val)
knn_val_accuracy = metrics.accuracy_score(y_val,knn_preds_val)
print('The validation confusion matrix for K-NN model is:', '\n', knn_val_cm)

print('Train accuracy: ',knn_train_accuracy,' | Validation accuracy: ',knn_val_accuracy)

The train confusion matrix for K-NN model is: 
 [[523   2]
 [ 35   3]]
The validation confusion matrix for K-NN model is: 
 [[1249    1]
 [  66    0]]
Train accuracy:  0.9342806394316163  | Validation accuracy:  0.9490881458966566


In [65]:
lr_train_cm = metrics.confusion_matrix(y_train, lr_preds_train)
lr_train_accuracy = metrics.accuracy_score(y_train,lr_preds_train)
print('The train confusion matrix for Logistic Regression model is:', '\n', lr_train_cm)

lr_val_cm = metrics.confusion_matrix(y_val,lr_preds_val)
lr_val_accuracy = metrics.accuracy_score(y_val,lr_preds_val)
print('The validation confusion matrix for Logistic Regression model is:', '\n', lr_val_cm)

print('Train accuracy: ',lr_train_accuracy,' | Validation accuracy: ',lr_val_accuracy)

The train confusion matrix for Logistic Regression model is: 
 [[523   2]
 [  3  35]]
The validation confusion matrix for Logistic Regression model is: 
 [[1244    6]
 [   5   61]]
Train accuracy:  0.9911190053285968  | Validation accuracy:  0.9916413373860182


In [66]:
accuracies = [dt_val_accuracy,knn_val_accuracy,lr_val_accuracy]
best_accuracy = np.max(accuracies)
accuracy_name = accuracies.index(best_accuracy)
print(accuracy_name,' has the best accuracy: ',best_accuracy)

2  has the best accuracy:  0.9916413373860182


#### => Logistic regression đang cho ra độ đo accuracy cao nhất trên tập validation

In [67]:
lr_pred_test = lr_model.predict(X_test)

# Create result DataFrame
result_df = pd.DataFrame({"fraud": lr_pred_test})

print('Rows with fraud predict: ',result_df[result_df['fraud'] == 1].shape)
print('Rows with non-fraud predict: ',result_df[result_df['fraud'] == 0].shape)

# Lưu kết quả ra file CSV
result_df.to_csv("./result/new_result.csv", index=False)
#

Rows with fraud predict:  (23810, 1)
Rows with non-fraud predict:  (474311, 1)


## Tìm kiếm siêu tham số

#### *Mô hình Decision Tree*

In [68]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, train_test_split

# Chia dữ liệu thành tập huấn luyện và tập validation
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Định nghĩa các giá trị siêu tham số cần tìm kiếm
param_grid = {
    'max_depth': [None, 10, 20, 30],  # Độ sâu tối đa của cây
    'min_samples_split': [2, 5, 10],   # Số lượng mẫu tối thiểu cần để chia một nút
    'min_samples_leaf': [1, 2, 4]      # Số lượng mẫu tối thiểu ở các lá
}

# Khởi tạo mô hình cây quyết định
dt_model = DecisionTreeClassifier()

# Tạo GridSearchCV
grid_search = GridSearchCV(estimator=dt_model, param_grid=param_grid, cv=5, scoring='accuracy')

# Tiến hành tìm kiếm siêu tham số
grid_search.fit(X_train, y_train)

# In ra kết quả tốt nhất
print("Best Parameters:", grid_search.best_params_)
print("Best Score:", grid_search.best_score_)

# Đánh giá mô hình tốt nhất trên tập validation
best_dt_model = grid_search.best_estimator_
val_preds = best_dt_model.predict(X_val)
print("Validation Accuracy:", metrics.accuracy_score(y_val, val_preds))


Best Parameters: {'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 2}
Best Score: 0.968888888888889
Validation Accuracy: 1.0


#### *Mô hình K-NN*

In [69]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier

# Khởi tạo mô hình
model = KNeighborsClassifier()

# Định nghĩa lưới các giá trị siêu tham số cần tìm kiếm
param_grid = {
    'n_neighbors': [3, 5, 7, 9],         # Số lượng hàng xóm gần nhất
    'weights': ['uniform', 'distance'],  # Phương pháp tính trọng số cho hàng xóm
    'metric': ['euclidean', 'manhattan'] # Phương pháp tính khoảng cách
}

# Khởi tạo GridSearchCV
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='accuracy')

# Thực hiện tìm kiếm siêu tham số
grid_search.fit(X_train, y_train)

# Lấy siêu tham số tốt nhất và điểm số tốt nhất
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("Best Parameters:", best_params)
print("Best Score:", best_score)

best_knn_model = grid_search.best_estimator_
val_preds = best_knn_model.predict(X_val)
print("Validation Accuracy:", metrics.accuracy_score(y_val, val_preds))

Best Parameters: {'metric': 'euclidean', 'n_neighbors': 7, 'weights': 'uniform'}
Best Score: 0.928888888888889
Validation Accuracy: 0.9469026548672567


#### *Mô hình Logistic Regression*

In [70]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

# Khởi tạo mô hình
model = LogisticRegression()

# Định nghĩa lưới các giá trị siêu tham số cần tìm kiếm
# param_grid = {
#     'penalty': ['l1', 'l2'],             # Phương pháp chuẩn hóa (L1 hoặc L2)
#     'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000, 100000, 1000000]  # Tham số điều chuẩn
# }
param_grid = {
    'penalty': ['l1', 'l2', 'elasticnet', 'none'],
    'C': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000],
    'solver': ['liblinear', 'saga'],
    'l1_ratio': [0.1, 0.5, 0.7, 0.9]  # Chỉ có hiệu lực khi penalty='elasticnet'
}
# Khởi tạo GridSearchCV
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='accuracy',n_jobs=-1)

# Thực hiện tìm kiếm siêu tham số
grid_search.fit(X_train, y_train)

# Lấy siêu tham số tốt nhất và điểm số tốt nhất
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("Best Parameters:", best_params)
print("Best Score:", best_score)

best_lr_model = grid_search.best_estimator_
val_preds = best_lr_model.predict(X_val)
print("Validation Accuracy:", metrics.accuracy_score(y_val, val_preds))

Best Parameters: {'C': 100, 'l1_ratio': 0.1, 'penalty': 'l1', 'solver': 'liblinear'}
Best Score: 0.9911111111111112
Validation Accuracy: 1.0


C:\Users\dangm\miniconda3\envs\hocmay\Lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
320 fits failed out of a total of 1280.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
160 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\dangm\miniconda3\envs\hocmay\Lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\dangm\miniconda3\envs\hocmay\Lib\site-packages\sklearn\linear_model\_logistic.py", line 1162, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\dangm\miniconda

In [71]:
lr_best_pred_test = best_lr_model.predict(X_test)

# Create result DataFrame
result_best_df = pd.DataFrame({"fraud": lr_best_pred_test})

print('Rows with fraud predict: ',result_best_df[result_best_df['fraud'] == 1].shape)
print('Rows with non-fraud predict: ',result_best_df[result_best_df['fraud'] == 0].shape)

# Lưu kết quả ra file CSV
result_df.to_csv("./result/new_result_with_hyperparameter.csv", index=False)
#

Rows with fraud predict:  (24834, 1)
Rows with non-fraud predict:  (473287, 1)


In [72]:
import pandas as pd

# Đọc tệp CSV
df = pd.read_csv("./result/DMC-2019-realclass-1.csv")

# Kiểm tra các cột hiện tại trong DataFrame
print("Columns before dropping:")
print(df.columns)
# Tạo danh sách các cột cần giữ lại
columns_to_keep = ['fraud', 'fraud5']

# Tạo danh sách các cột cần xóa bằng cách lấy tất cả các cột trừ cột cần giữ lại
columns_to_drop = [col for col in df.columns if col not in columns_to_keep]

# Xóa các cột không cần thiết
df = df.drop(columns=columns_to_drop)

# Kiểm tra lại các cột sau khi xóa
print("Columns after dropping:")
print(df.columns)

# Lưu DataFrame đã cập nhật vào tệp CSV mới
df.to_csv("./result/DMC-2019-realclass-1.csv", index=False)


Columns before dropping:
Index(['fraud', 'fraud5'], dtype='object')
Columns after dropping:
Index(['fraud', 'fraud5'], dtype='object')


In [73]:
import pandas as pd

# Giả sử bạn đã có các giá trị dự đoán từ mô hình logistic regression
# lr_best_pred_test = best_lr_model.predict(X_test)

# Tạo DataFrame chứa kết quả dự đoán
result_best_df = pd.DataFrame({"fraud5": lr_best_pred_test})

# In thông tin về các dự đoán
print('Rows with fraud predict: ', result_best_df[result_best_df['fraud5'] == 1].shape)
print('Rows with non-fraud predict: ', result_best_df[result_best_df['fraud5'] == 0].shape)

# Đọc tệp DMC-2019-realclass-1.csv
original_df = pd.read_csv("./result/DMC-2019-realclass-1.csv")

# Kiểm tra xem số lượng hàng có khớp nhau không
assert len(original_df) == len(result_best_df), "Số lượng hàng không khớp giữa hai DataFrame"

# Thêm cột fraud3 vào DataFrame ban đầu
original_df['fraud5'] = result_best_df['fraud5']

# Lưu DataFrame đã cập nhật vào tệp CSV mới
original_df.to_csv("./result/DMC-2019-realclass-1.csv", index=False)


Rows with fraud predict:  (24834, 1)
Rows with non-fraud predict:  (473287, 1)


In [74]:
import pandas as pd
from sklearn.metrics import confusion_matrix

# Đọc dữ liệu từ tệp CSV
df = pd.read_csv("./result/DMC-2019-realclass-1.csv")

# Trích xuất các giá trị thực tế và dự đoán
y_actual = df['fraud'].values
y_predict = df['fraud5'].values

# Tính toán ma trận nhầm lẫn
cm = confusion_matrix(y_actual, y_predict)


# Định nghĩa ma trận chi phí
cost_matrix = np.array([[0, -5],
                        [-25, 5]])

# Tính toán lợi ích tổng thể
benefit = np.sum(cm * cost_matrix)

# In kết quả
print("Confusion Matrix:")
print(cm)
print("Benefit:", benefit)

Confusion Matrix:
[[471477   2917]
 [  1810  21917]]
Benefit: 49750


#### **So sánh số dự đoán giữa 2 mô hình normal và hyper-parameter applied**

In [75]:

print('Rows end up detecting fraud with normal model: ',len(result_df[result_df['fraud'] == 1]))
print('Rows end up detecting fraud with hyper-parameter applied model: ',len(result_best_df[result_best_df['fraud5'] == 1]))

Rows end up detecting fraud with normal model:  23810
Rows end up detecting fraud with hyper-parameter applied model:  24834
